In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml.html as lh
from PIL import Image
import urllib.request
import urllib
import pickle

In [3]:
#link = "http://www.eng.usf.edu/cvprg/Mammography/DDSM/thumbnails/normals/normal_03/case0330/A-0330-1.html"

In [4]:
#r = requests.get(link)
#soup = BeautifulSoup(r.text)

#for image_src in soup.find_all("img"):
#    image_url = image_src['src']
#    print(image_url)
#    full_url = "http://www.eng.usf.edu/cvprg/Mammography/DDSM/thumbnails/normals/normal_03/case0330/"+image_url
#    img = Image.open(requests.get(full_url, stream = True).raw)
#    img.save(image_url)

----------------

## Scraping full DDSM

In [2]:
url = 'http://www.eng.usf.edu/cvprg/Mammography/Database.html'

### Main page

In [3]:
page = requests.get(url)#Store the contents of the website under doc
html = BeautifulSoup(page.text)
table = html.find('table')
links = table.findAll('a')

In [4]:
links[0]['href']

'DDSM/thumbnails/normals/normal_01/overview.html'

In [5]:
ftps = []
table_content = {}
table_normal = []
table_cancer = []
table_benign = []

for link in links:
    
    if 'thumbnails' in link:
        thumbnail_url = link['href']
        case_type = thumbnail_url[16:].split("/",1)[0]
        thumbnail_url = "http://www.eng.usf.edu/cvprg/Mammography/"+thumbnail_url
        main_name = thumbnail_url[:-14]
        
        th_page = requests.get(thumbnail_url) #Store the contents of the website under doc
        th_html = BeautifulSoup(th_page.text)
        th_table = th_html.find('table')
        th_links = th_table.findAll('a')
        
        case_id = 0
        
        for th_link in th_links:
            
            case_url = th_link['href']
            case_url = main_name+"/"+case_url
            case_url_page = requests.get(case_url) #Store the contents of the website under doc
            case_url_html = BeautifulSoup(case_url_page.text)
            case_url_tables = case_url_html.findAll('table')
            
            for i in range(2,len(case_url_tables)):
                
                info = case_url_tables[i].findAll(lambda tag: tag.name=='pre')
                
                if case_type=='normals':
                    table_normal.append([i, case_id, info])
                
                if case_type=='cancers':
                    table_cancer.append([i, case_id, info])
                
                if case_type=='benigns':
                    table_benign.append([i, case_id, info])
        
        #table_content[case_type]
            
            case_id+=1
        
        
    if 'ftp' in link:
        ftp_url = link['href']
        ftps.append(ftp_url)

In [6]:
len(table_normal)

695

In [7]:
len(table_cancer)/2

914.0

In [8]:
len(table_benign)/2

855.0

In [9]:
with open('ftps_list.pkl', 'wb') as f:
    pickle.dump(ftps, f)

## Statystyki

In [10]:
count_spic = 0
count_arch = 0

for elem in table_benign:
    if elem[0]!=2:
        if 'SPICULATED' in str(elem[2]):
            count_spic += 1
            
        if 'ARCHITECTURAL_DISTORTION' in str(elem[2]):
            count_arch += 1

In [11]:
count_spic_canc = 0
count_arch_canc = 0

for elem in table_cancer:
    if elem[0]!=2:
        if 'SPICULATED' in str(elem[2]):
            count_spic_canc += 1
            
        if 'ARCHITECTURAL_DISTORTION' in str(elem[2]):
            count_arch_canc += 1

In [12]:
print(count_arch_canc)
print(count_spic_canc)
print(count_arch)
print(count_spic)

113
292
17
20


-----------

## Tabelki

### Normals

In [13]:
table_normal

[[2, 0, [<pre>
   ics_version 1.0
   filename A-0002-1
   DATE_OF_STUDY 5 3 1991
   PATIENT_AGE 63
   FILM
   FILM_TYPE REGULAR
   DENSITY 2
   DATE_DIGITIZED 2 11 1995
   DIGITIZER DBA 21
   
   LEFT_CC LINES 4349 PIXELS_PER_LINE 1979 BITS_PER_PIXEL 16 RESOLUTION 42 NON_OVERLAY
   LEFT_MLO LINES 5069 PIXELS_PER_LINE 2039 BITS_PER_PIXEL 16 RESOLUTION 42 NON_OVERLAY
   RIGHT_CC LINES 4334 PIXELS_PER_LINE 1874 BITS_PER_PIXEL 16 RESOLUTION 42 NON_OVERLAY
   RIGHT_MLO LINES 5084 PIXELS_PER_LINE 2129 BITS_PER_PIXEL 16 RESOLUTION 42 NON_OVERLAY
   </pre>]], [2, 1, [<pre>
   ics_version 1.0
   filename A-0003-1
   DATE_OF_STUDY 7 3 1991
   PATIENT_AGE 66
   FILM
   FILM_TYPE REGULAR
   DENSITY 3
   DATE_DIGITIZED 10 8 1995
   DIGITIZER DBA 21
   
   LEFT_CC LINES 4514 PIXELS_PER_LINE 2084 BITS_PER_PIXEL 16 RESOLUTION 42 NON_OVERLAY
   LEFT_MLO LINES 5159 PIXELS_PER_LINE 2069 BITS_PER_PIXEL 16 RESOLUTION 42 NON_OVERLAY
   RIGHT_CC LINES 4679 PIXELS_PER_LINE 2114 BITS_PER_PIXEL 16 RESOLUTION 42

In [14]:
table_benign[6]

[2, 3, [<pre>
  ics_version 1.0
  filename C-0235-1
  DATE_OF_STUDY 5 1 1995
  PATIENT_AGE 42
  FILM
  FILM_TYPE REGULAR
  DENSITY 2
  DATE_DIGITIZED 28 4 1998
  DIGITIZER LUMISYS LASER
  SEQUENCE
  LEFT_CC LINES 6024 PIXELS_PER_LINE 3232 BITS_PER_PIXEL 12 RESOLUTION 50 NON_OVERLAY
  LEFT_MLO LINES 5968 PIXELS_PER_LINE 3312 BITS_PER_PIXEL 12 RESOLUTION 50 NON_OVERLAY
  RIGHT_CC LINES 5912 PIXELS_PER_LINE 3104 BITS_PER_PIXEL 12 RESOLUTION 50 OVERLAY
  RIGHT_MLO LINES 5960 PIXELS_PER_LINE 3096 BITS_PER_PIXEL 12 RESOLUTION 50 OVERLAY
  </pre>]]

In [19]:
normals_df = pd.DataFrame()

for elem in table_normal:
    if elem[0]==2:
        #print(elem[2])
        
        dictionary = {}
        
        for line in str(elem[2]).splitlines():
            #print(line)
            word_list = line.split()
            if len(word_list)>0:
                #print(word_list[0])
                #print(word_list[1:])
                dictionary[word_list[0]] = ' '.join(word_list[1:]) 
    
        normals_df = normals_df.append(dictionary, ignore_index=True)

In [20]:
normals_df.head()

,</pre>],DATE_DIGITIZED,DATE_OF_STUDY,DENSITY,DIGITIZER,FILM,FILM_TYPE,LEFT_CC,LEFT_MLO,PATIENT_AGE,RIGHT_CC,RIGHT_MLO,[<pre>,filename,ics_version,SEQUENCE
0,,2 11 1995,5 3 1991,2,DBA 21,,REGULAR,LINES 4349 PIXELS_PER_LINE 1979 BITS_PER_PIXEL...,LINES 5069 PIXELS_PER_LINE 2039 BITS_PER_PIXEL...,63,LINES 4334 PIXELS_PER_LINE 1874 BITS_PER_PIXEL...,LINES 5084 PIXELS_PER_LINE 2129 BITS_PER_PIXEL...,,A-0002-1,1.0,NaN
1,,10 8 1995,7 3 1991,3,DBA 21,,REGULAR,LINES 4514 PIXELS_PER_LINE 2084 BITS_PER_PIXEL...,LINES 5159 PIXELS_PER_LINE 2069 BITS_PER_PIXEL...,66,LINES 4679 PIXELS_PER_LINE 2114 BITS_PER_PIXEL...,LINES 5399 PIXELS_PER_LINE 2159 BITS_PER_PIXEL...,,A-0003-1,1.0,NaN
2,,15 8 1995,23 10 1991,4,DBA 21,,REGULAR,LINES 3959 PIXELS_PER_LINE 1799 BITS_PER_PIXEL...,LINES 4454 PIXELS_PER_LINE 1949 BITS_PER_PIXEL...,44,LINES 4139 PIXELS_PER_LINE 1904 BITS_PER_PIXEL...,LINES 5099 PIXELS_PER_LINE 2084 BITS_PER_PIXEL...,,A-0005-1,1.0,NaN
3,,17 8 1995,25 3 1991,2,DBA 21,,REGULAR,LINES 4874 PIXELS_PER_LINE 2429 BITS_PER_PIXEL...,LINES 4979 PIXELS_PER_LINE 2684 BITS_PER_PIXEL...,55,LINES 4994 PIXELS_PER_LINE 2309 BITS_PER_PIXEL...,LINES 5459 PIXELS_PER_LINE 2489 BITS_PER_PIXEL...,,A-0006-1,1.0,NaN
4,,22 8 1995,8 4 1991,4,DBA 21,,REGULAR,LINES 5534 PIXELS_PER_LINE 2969 BITS_PER_PIXEL...,LINES 5594 PIXELS_PER_LINE 3224 BITS_PER_PIXEL...,63,LINES 5339 PIXELS_PER_LINE 2684 BITS_PER_PIXEL...,LINES 5594 PIXELS_PER_LINE 2954 BITS_PER_PIXEL...,,A-0009-1,1.0,NaN


In [21]:
with open('normals_df.pkl', 'wb') as f:
    pickle.dump(normals_df, f)

### Cancers

In [30]:
table_cancer[0:4]

[[2, 0, [<pre>
   ics_version 1.0
   filename C-0001-1
   DATE_OF_STUDY 15 10 1992
   PATIENT_AGE 65
   FILM
   FILM_TYPE REGULAR
   DENSITY 2
   DATE_DIGITIZED 18 8 1997
   DIGITIZER LUMISYS LASER
   SEQUENCE
   LEFT_CC LINES 4608 PIXELS_PER_LINE 2928 BITS_PER_PIXEL 12 RESOLUTION 50 NON_OVERLAY
   LEFT_MLO LINES 4592 PIXELS_PER_LINE 2896 BITS_PER_PIXEL 12 RESOLUTION 50 NON_OVERLAY
   RIGHT_CC LINES 4584 PIXELS_PER_LINE 2920 BITS_PER_PIXEL 12 RESOLUTION 50 OVERLAY
   RIGHT_MLO LINES 4616 PIXELS_PER_LINE 2888 BITS_PER_PIXEL 12 RESOLUTION 50 OVERLAY
   </pre>]], [3, 0, [<pre>
   FILE: C_0001_1.RIGHT_MLO.OVERLAY
   
   TOTAL_ABNORMALITIES 1
   <font color="FF0000">ABNORMALITY 1
   LESION_TYPE MASS SHAPE IRREGULAR MARGINS ILL_DEFINED
   ASSESSMENT 4
   SUBTLETY 3
   PATHOLOGY MALIGNANT
   TOTAL_OUTLINES 1 
   BOUNDARY
   </font></pre>, <pre>
   FILE: C_0001_1.RIGHT_CC.OVERLAY
   
   TOTAL_ABNORMALITIES 1
   <font color="FF0000">ABNORMALITY 1
   LESION_TYPE MASS SHAPE IRREGULAR MARGINS SPIC

In [31]:
count=0
for elem in table_cancer:
    print(elem[0])
    print(elem[1])
    print("----")
    count+=1
    if count>10:
        break;

2
0
----
3
0
----
2
1
----
3
1
----
2
2
----
3
2
----
2
3
----
3
3
----
2
4
----
3
4
----
2
5
----


In [24]:
cancers_df = pd.DataFrame()
cancers_abnormalities_df = pd.DataFrame()

for elem in table_cancer:
    if elem[0]==2:
        #print(elem[2])
        
        dictionary = {}
        
        for line in str(elem[2]).splitlines():
            #print(line)
            word_list = line.split()
            if len(word_list)>0:
                #print(word_list[0])
                #print(word_list[1:])
                dictionary[word_list[0]] = ' '.join(word_list[1:]) 
    
        cancers_df = cancers_df.append(dictionary, ignore_index=True)
        
    if elem[0]==3:
        
        dictionary = {}
        
        for line in str(elem[2]).splitlines():
            #print(line)
            word_list = line.split()
            if len(word_list)>0:
                #print(word_list[0])
                #print(word_list[1:])
                dictionary[word_list[0]] = ' '.join(word_list[1:]) 
    
        cancers_abnormalities_df = cancers_abnormalities_df.append(dictionary, ignore_index=True)
        

In [27]:
cancers_df.head()

,</pre>],DATE_DIGITIZED,DATE_OF_STUDY,DENSITY,DIGITIZER,FILM,FILM_TYPE,LEFT_CC,LEFT_MLO,PATIENT_AGE,RIGHT_CC,RIGHT_MLO,SEQUENCE,[<pre>,filename,ics_version
0,,18 8 1997,15 10 1992,2,LUMISYS LASER,,REGULAR,LINES 4608 PIXELS_PER_LINE 2928 BITS_PER_PIXEL...,LINES 4592 PIXELS_PER_LINE 2896 BITS_PER_PIXEL...,65,LINES 4584 PIXELS_PER_LINE 2920 BITS_PER_PIXEL...,LINES 4616 PIXELS_PER_LINE 2888 BITS_PER_PIXEL...,,,C-0001-1,1.0
1,,18 8 1997,5 11 1992,2,LUMISYS LASER,,REGULAR,LINES 5928 PIXELS_PER_LINE 3776 BITS_PER_PIXEL...,LINES 5824 PIXELS_PER_LINE 4104 BITS_PER_PIXEL...,72,LINES 5704 PIXELS_PER_LINE 4120 BITS_PER_PIXEL...,LINES 5792 PIXELS_PER_LINE 4144 BITS_PER_PIXEL...,,,C-0002-1,1.0
2,,18 8 1997,2 8 1992,2,LUMISYS LASER,,REGULAR,LINES 5896 PIXELS_PER_LINE 4088 BITS_PER_PIXEL...,LINES 5832 PIXELS_PER_LINE 4104 BITS_PER_PIXEL...,46,LINES 5928 PIXELS_PER_LINE 3824 BITS_PER_PIXEL...,LINES 5960 PIXELS_PER_LINE 4032 BITS_PER_PIXEL...,,,C-0003-1,1.0
3,,18 8 1997,20 11 1992,3,LUMISYS LASER,,REGULAR,LINES 4704 PIXELS_PER_LINE 3000 BITS_PER_PIXEL...,LINES 4584 PIXELS_PER_LINE 2936 BITS_PER_PIXEL...,75,LINES 4712 PIXELS_PER_LINE 2944 BITS_PER_PIXEL...,LINES 4648 PIXELS_PER_LINE 2968 BITS_PER_PIXEL...,,,C-0004-1,1.0
4,,10 9 1997,23 10 1992,2,LUMISYS LASER,,REGULAR,LINES 4648 PIXELS_PER_LINE 2800 BITS_PER_PIXEL...,LINES 4696 PIXELS_PER_LINE 2888 BITS_PER_PIXEL...,61,LINES 4688 PIXELS_PER_LINE 2704 BITS_PER_PIXEL...,LINES 4720 PIXELS_PER_LINE 2760 BITS_PER_PIXEL...,,,C-0006-1,1.0


In [28]:
cancers_abnormalities_df.head()

,"</font></pre>,",</font></pre>],<font,ASSESSMENT,BOUNDARY,FILE:,LESION_TYPE,PATHOLOGY,SUBTLETY,TOTAL_ABNORMALITIES,TOTAL_OUTLINES,[<pre>,</font><font,CORE
0,<pre>,,"color=""FF0000"">ABNORMALITY 1",5,,C_0001_1.RIGHT_CC.OVERLAY,MASS SHAPE IRREGULAR MARGINS SPICULATED,MALIGNANT,5,1,1,,NaN,NaN
1,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0002_1.LEFT_CC.OVERLAY,CALCIFICATION TYPE PLEOMORPHIC DISTRIBUTION SE...,MALIGNANT,2,1,1,,NaN,NaN
2,<pre>,,"color=""FF0000"">ABNORMALITY 1",5,,C_0003_1.RIGHT_CC.OVERLAY,MASS SHAPE IRREGULAR MARGINS SPICULATED,MALIGNANT,5,1,1,,NaN,NaN
3,<pre>,,"color=""FF0000"">ABNORMALITY 1",5,,C_0004_1.RIGHT_CC.OVERLAY,MASS SHAPE IRREGULAR MARGINS SPICULATED,MALIGNANT,5,1,1,,NaN,NaN
4,<pre>,,"color=""FF0000"">ABNORMALITY 1",5,,C_0006_1.RIGHT_CC.OVERLAY,MASS SHAPE IRREGULAR MARGINS SPICULATED,MALIGNANT,5,1,1,,NaN,NaN


In [32]:
with open('cancers_df.pkl', 'wb') as f:
    pickle.dump(cancers_df, f)

In [33]:
with open('cancers_abnormalities_df.pkl', 'wb') as f:
    pickle.dump(cancers_abnormalities_df, f)

### Benigns

In [34]:
count=0
for elem in table_benign:
    print(elem[0])
    print(elem[1])
    print("----")
    count+=1
    if count>10:
        break;

2
0
----
3
0
----
2
1
----
3
1
----
2
2
----
3
2
----
2
3
----
3
3
----
2
4
----
3
4
----
2
5
----


In [36]:
benigns_df = pd.DataFrame()
benigns_abnormalities_df = pd.DataFrame()

for elem in table_benign:
    if elem[0]==2:
        #print(elem[2])
        
        dictionary = {}
        
        for line in str(elem[2]).splitlines():
            #print(line)
            word_list = line.split()
            if len(word_list)>0:
                #print(word_list[0])
                #print(word_list[1:])
                dictionary[word_list[0]] = ' '.join(word_list[1:]) 
    
        benigns_df = benigns_df.append(dictionary, ignore_index=True)
        
    if elem[0]==3:
        
        dictionary = {}
        
        for line in str(elem[2]).splitlines():
            #print(line)
            word_list = line.split()
            if len(word_list)>0:
                #print(word_list[0])
                #print(word_list[1:])
                dictionary[word_list[0]] = ' '.join(word_list[1:]) 
    
        benigns_abnormalities_df = benigns_abnormalities_df.append(dictionary, ignore_index=True)
        

In [37]:
benigns_df.head()

,</pre>],DATE_DIGITIZED,DATE_OF_STUDY,DENSITY,DIGITIZER,FILM,FILM_TYPE,LEFT_CC,LEFT_MLO,PATIENT_AGE,RIGHT_CC,RIGHT_MLO,SEQUENCE,[<pre>,filename,ics_version
0,,9 10 1997,2 3 1993,3,LUMISYS LASER,,REGULAR,LINES 4648 PIXELS_PER_LINE 2672 BITS_PER_PIXEL...,LINES 4680 PIXELS_PER_LINE 2664 BITS_PER_PIXEL...,66,LINES 4704 PIXELS_PER_LINE 2816 BITS_PER_PIXEL...,LINES 4648 PIXELS_PER_LINE 2864 BITS_PER_PIXEL...,,,C-0029-1,1.0
1,,10 10 1997,18 11 1993,3,LUMISYS LASER,,REGULAR,LINES 4672 PIXELS_PER_LINE 2616 BITS_PER_PIXEL...,LINES 4664 PIXELS_PER_LINE 2424 BITS_PER_PIXEL...,60,LINES 4656 PIXELS_PER_LINE 2680 BITS_PER_PIXEL...,LINES 4680 PIXELS_PER_LINE 2752 BITS_PER_PIXEL...,,,C-0033-1,1.0
2,,14 4 1998,23 2 1996,2,LUMISYS LASER,,REGULAR,LINES 6000 PIXELS_PER_LINE 3264 BITS_PER_PIXEL...,LINES 5992 PIXELS_PER_LINE 3368 BITS_PER_PIXEL...,56,LINES 5960 PIXELS_PER_LINE 3536 BITS_PER_PIXEL...,LINES 5944 PIXELS_PER_LINE 3752 BITS_PER_PIXEL...,,,C-0217-1,1.0
3,,28 4 1998,5 1 1995,2,LUMISYS LASER,,REGULAR,LINES 6024 PIXELS_PER_LINE 3232 BITS_PER_PIXEL...,LINES 5968 PIXELS_PER_LINE 3312 BITS_PER_PIXEL...,42,LINES 5912 PIXELS_PER_LINE 3104 BITS_PER_PIXEL...,LINES 5960 PIXELS_PER_LINE 3096 BITS_PER_PIXEL...,,,C-0235-1,1.0
4,,28 4 1998,19 8 1994,3,LUMISYS LASER,,REGULAR,LINES 4736 PIXELS_PER_LINE 2776 BITS_PER_PIXEL...,LINES 4752 PIXELS_PER_LINE 2552 BITS_PER_PIXEL...,62,LINES 4720 PIXELS_PER_LINE 2720 BITS_PER_PIXEL...,LINES 4704 PIXELS_PER_LINE 2840 BITS_PER_PIXEL...,,,C-0237-1,1.0


In [38]:
benigns_abnormalities_df

,"</font></pre>,",</font></pre>],<font,ASSESSMENT,BOUNDARY,FILE:,LESION_TYPE,PATHOLOGY,SUBTLETY,TOTAL_ABNORMALITIES,TOTAL_OUTLINES,[<pre>,</font><font,CORE,CLUSTERED
0,<pre>,,"color=""FF0000"">ABNORMALITY 1",3,,C_0029_1.LEFT_CC.OVERLAY,MASS SHAPE OVAL MARGINS ILL_DEFINED,BENIGN,3,1,1,,NaN,NaN,NaN
1,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0033_1.RIGHT_CC.OVERLAY,MASS SHAPE OVAL MARGINS MICROLOBULATED,BENIGN,4,1,1,,NaN,NaN,NaN
2,<pre>,,"color=""FF0000"">ABNORMALITY 1",3,,C_0217_1.RIGHT_CC.OVERLAY,MASS SHAPE ROUND MARGINS CIRCUMSCRIBED,BENIGN,5,1,1,,NaN,NaN,NaN
3,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0235_1.RIGHT_CC.OVERLAY,MASS SHAPE OVAL MARGINS MICROLOBULATED,BENIGN,5,1,1,,NaN,NaN,NaN
4,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0237_1.LEFT_CC.OVERLAY,MASS SHAPE ROUND MARGINS CIRCUMSCRIBED,BENIGN,4,1,1,,NaN,NaN,NaN
5,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0238_1.LEFT_CC.OVERLAY,CALCIFICATION TYPE AMORPHOUS DISTRIBUTION CLUS...,BENIGN,5,1,1,,NaN,NaN,NaN
6,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0239_1.LEFT_CC.OVERLAY,CALCIFICATION TYPE FINE_LINEAR_BRANCHING DISTR...,BENIGN,5,1,1,,NaN,NaN,NaN
7,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0240_1.RIGHT_CC.OVERLAY,MASS SHAPE OVAL MARGINS ILL_DEFINED,BENIGN,3,1,1,,NaN,NaN,NaN
8,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0241_1.LEFT_CC.OVERLAY,MASS SHAPE IRREGULAR MARGINS ILL_DEFINED,BENIGN,3,1,1,,NaN,NaN,NaN
9,<pre>,,"color=""FF0000"">ABNORMALITY 1",4,,C_0242_1.RIGHT_CC.OVERLAY,MASS SHAPE ARCHITECTURAL_DISTORTION MARGINS SP...,BENIGN,5,1,1,,NaN,NaN,NaN


In [41]:
with open('benigns_df.pkl', 'wb') as f:
    pickle.dump(benigns_df, f)

In [42]:
with open('benigns_abnormalities_df.pkl', 'wb') as f:
    pickle.dump(benigns_abnormalities_df, f)

-----

## Obrazy

In [73]:
ftps[0:10]

['ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_01',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_02',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_03',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_04',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_05',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_06',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_07',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_08',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_09',
 'ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_10']

In [17]:
for ftp_url in ftps:

    with urllib.request.urlopen(ftp_url) as r:
        data = r.read()

    links = [
        ftp_url + '/' + line.split()[-1] 
        for line in data.decode().splitlines()
    ]
    
    for link in links:
        print(link)
        
        with urllib.request.urlopen(link) as r:
            new_data = r.read()
        
        new_links = [
        link + '/' + line.split()[-1] 
        for line in new_data.decode().splitlines()
        ]
        
        for file in new_links:
            if "LJPEG" in str(file):
                print(file)
                
                print(str(file).split('/')[-1].split('_')[1])
                
                #print([int(s) for s in str(file).split('/')[-1].split('_') if s.isdigit()])
                
                #print(str(file).split('/')[-1])
                urllib.request.urlretrieve(file, '/home/kasia/Pulpit/MGR/scraped_data/'+str(file).split('/')[-1])


ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_01/case0002
ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_01/case0002/A_0002_1.LEFT_CC.LJPEG
0002
ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_01/case0002/A_0002_1.LEFT_MLO.LJPEG
0002
ftp://figment.csee.usf.edu/pub/DDSM/cases/normals/normal_01/case0002/A_0002_1.RIGHT_CC.LJPEG
0002


KeyboardInterrupt: 

### Thumbnails

In [52]:
th_url = thumbnails[0]

th_page = requests.get(th_url)#Store the contents of the website under doc
th_html = BeautifulSoup(page.text)
th_table = html.find('table')
th_links = table.findAll('a')

In [60]:
cases = []

for th_link in th_links:
    
    if 'thumbnails' in link:
        thumbnail_url = link['href']
        thumbnail_url = "http://www.eng.usf.edu/cvprg/Mammography/"+thumbnail_url
        thumbnails.append(thumbnail_url)
        
    if 'ftp' in link:
        ftp_url = link['href']
        ftps.append(ftp_url)

In [7]:
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [9]:
tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"VOLUME"
2:"CASES"
3:"SIZE"
4:"SCANNER"
5:"BITS"
6:"RESOLUTION"
7:"THUMBNAILS"
8:"NOTES"
9:"AVAILABILITY"


In [19]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    #if len(T)!=10:
    #    break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        print(data)
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

normal_01
111
5.8 GB
DBA
16
42 microns
thumbnails
notes
ftp 
normal_02
117
6.6 GB
DBA
16
42 microns
thumbnails
notes
ftp 
normal_03
38
4.1 GB
DBA
16
42 microns
thumbnails
notes
ftp 
normal_04
57
5.1 GB
DBA
16
42 microns
thumbnails
notes
ftp 
normal_05
47
4.3 GB
DBA
16
42 microns
thumbnails
notes
ftp 
normal_06
60
5.5 GB
DBA
16
42 microns
thumbnails
notes
ftp 
normal_07
78
6.2 GB
HOWTEK
12
43.5 microns
thumbnails
notes
ftp 
normal_08
27
2.8 GB
HOWTEK
12
43.5 microns
thumbnails
notes
ftp 
normal_09
59
4.9 GB
LUMYSIS
12
50 microns
thumbnails
notes
ftp 
normal_10
23
2.1 GB
LUMYSIS
12
50 microns
thumbnails
notes
ftp 
normal_11
58
6.1 GB
HOWTEK
12
43.5 microns
thumbnails
notes
ftp 
normal_12
20
2.2 GB
HOWTEK
12
43.5 microns
thumbnails
notes
ftp 
cancer_01
69
3.9 GB
LUMISYS
12
50 microns
thumbnails
notes
ftp 
cancer_02
88
5.7 GB
LUMISYS
12
50 microns
thumbnails
notes
ftp 
cancer_03
66
6.0 GB
DBA
16
42 microns
thumbnails
notes
ftp 
cancer_04
31
2.8 GB
DBA
16
42 microns
thumbnails
notes
ftp 
ca